Import Library 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score ,accuracy_score ,precision_recall_fscore_support
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

C:\Users\admin\AppData\Local\Temp\ipykernel_3308\3348616918.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


load dataset

In [2]:
a1 = pd.read_excel("../data/case_study1.xlsx")
a2 = pd.read_excel("../data/case_study2.xlsx")

In [3]:
df1 = a1.copy()
df2 = a2.copy()

In [4]:
df1.head(3)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.2,0.8,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.0,0.0,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.0,0.0,...,0,6,1,0,0,2,6,0,47,2


In [5]:
df2.head(3)

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PROSPECTID            51336 non-null  int64  
 1   Total_TL              51336 non-null  int64  
 2   Tot_Closed_TL         51336 non-null  int64  
 3   Tot_Active_TL         51336 non-null  int64  
 4   Total_TL_opened_L6M   51336 non-null  int64  
 5   Tot_TL_closed_L6M     51336 non-null  int64  
 6   pct_tl_open_L6M       51336 non-null  float64
 7   pct_tl_closed_L6M     51336 non-null  float64
 8   pct_active_tl         51336 non-null  float64
 9   pct_closed_tl         51336 non-null  float64
 10  Total_TL_opened_L12M  51336 non-null  int64  
 11  Tot_TL_closed_L12M    51336 non-null  int64  
 12  pct_tl_open_L12M      51336 non-null  float64
 13  pct_tl_closed_L12M    51336 non-null  float64
 14  Tot_Missed_Pmnt       51336 non-null  int64  
 15  Auto_TL            

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51336 entries, 0 to 51335
Data columns (total 62 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PROSPECTID                    51336 non-null  int64  
 1   time_since_recent_payment     51336 non-null  int64  
 2   time_since_first_deliquency   51336 non-null  int64  
 3   time_since_recent_deliquency  51336 non-null  int64  
 4   num_times_delinquent          51336 non-null  int64  
 5   max_delinquency_level         51336 non-null  int64  
 6   max_recent_level_of_deliq     51336 non-null  int64  
 7   num_deliq_6mts                51336 non-null  int64  
 8   num_deliq_12mts               51336 non-null  int64  
 9   num_deliq_6_12mts             51336 non-null  int64  
 10  max_deliq_6mts                51336 non-null  int64  
 11  max_deliq_12mts               51336 non-null  int64  
 12  num_times_30p_dpd             51336 non-null  int64  
 13  n

In [8]:
df1.shape ,df2.shape

((51336, 26), (51336, 62))

In [9]:
#Remove Null values from df1
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]

In [10]:
#shape after removing null values
df1.shape

(51296, 26)

In [11]:
#Removing null values from df2
cols_t_remv = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        cols_t_remv.append(i)

In [12]:
cols_t_remv

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [13]:
#dropping columns which have more than 10000 null values
df2 = df2.drop(cols_t_remv ,axis=1)

In [14]:
#shape after dropping
df2.shape

(51336, 54)

In [15]:
for i in df2.columns:
    df2 = df2.loc[df2[i] != -99999]

In [16]:
df2.isna().sum()

PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                       0
num_std_6mts                  0
num_std_12mts                 0
num_sub                       0
num_sub_6mts                  0
num_sub_12mts                 0
num_dbt                       0
num_dbt_6mts                  0
num_dbt_12mts                 0
num_lss                       0
num_lss_6mts                  0
num_lss_12mts                 0
recent_level_of_deliq         0
tot_enq                       0
CC_enq                        0
CC_enq_L6m                    0
CC_enq_L12m                   0
PL_enq                        0
PL_enq_L6m                    0
PL_enq_L12m                   0
time_since_recent_enq         0
enq_L12m                      0
enq_L6m 

In [17]:
df1.isna().sum()

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64

In [18]:
#merge df1 and df2
#checking common columns in dataframe 1 and 2
for i in df1.columns:
    if i in df2.columns:
        print(i)

PROSPECTID


In [19]:
#create merged dataframe
df = pd.merge( df1 ,df2 ,how="inner" ,left_on=['PROSPECTID'] ,right_on=['PROSPECTID'])

In [20]:
df.head(2)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.0,0.0,0.2,0.8,...,0.0,0.0,0.0,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2


In [21]:
df.isna().sum()

PROSPECTID             0
Total_TL               0
Tot_Closed_TL          0
Tot_Active_TL          0
Total_TL_opened_L6M    0
                      ..
GL_Flag                0
last_prod_enq2         0
first_prod_enq2        0
Credit_Score           0
Approved_Flag          0
Length: 79, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [23]:
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [24]:
for i in df.columns:
    if df[i].dtype != 'object':
        print(i)

PROSPECTID
Total_TL
Tot_Closed_TL
Tot_Active_TL
Total_TL_opened_L6M
Tot_TL_closed_L6M
pct_tl_open_L6M
pct_tl_closed_L6M
pct_active_tl
pct_closed_tl
Total_TL_opened_L12M
Tot_TL_closed_L12M
pct_tl_open_L12M
pct_tl_closed_L12M
Tot_Missed_Pmnt
Auto_TL
CC_TL
Consumer_TL
Gold_TL
Home_TL
PL_TL
Secured_TL
Unsecured_TL
Other_TL
Age_Oldest_TL
Age_Newest_TL
time_since_recent_payment
num_times_delinquent
max_recent_level_of_deliq
num_deliq_6mts
num_deliq_12mts
num_deliq_6_12mts
num_times_30p_dpd
num_times_60p_dpd
num_std
num_std_6mts
num_std_12mts
num_sub
num_sub_6mts
num_sub_12mts
num_dbt
num_dbt_6mts
num_dbt_12mts
num_lss
num_lss_6mts
num_lss_12mts
recent_level_of_deliq
tot_enq
CC_enq
CC_enq_L6m
CC_enq_L12m
PL_enq
PL_enq_L6m
PL_enq_L12m
time_since_recent_enq
enq_L12m
enq_L6m
enq_L3m
AGE
NETMONTHLYINCOME
Time_With_Curr_Empr
pct_of_active_TLs_ever
pct_opened_TLs_L6m_of_L12m
pct_currentBal_all_TL
CC_Flag
PL_Flag
pct_PL_enq_L6m_of_L12m
pct_CC_enq_L6m_of_L12m
pct_PL_enq_L6m_of_ever
pct_CC_enq_L6m_of_

In [25]:
df['MARITALSTATUS'].value_counts()

MARITALSTATUS
Married    30886
Single     11178
Name: count, dtype: int64

In [26]:
df['last_prod_enq2'].value_counts()

last_prod_enq2
ConsumerLoan    16480
others          13653
PL               7553
CC               2195
AL               1353
HL                830
Name: count, dtype: int64

In [27]:
from scipy.stats import chi2_contingency

for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '----', pval)


MARITALSTATUS ---- 3.578180861038862e-233
EDUCATION ---- 2.6942265249737532e-30
GENDER ---- 1.907936100186563e-05
last_prod_enq2 ---- 0.0
first_prod_enq2 ---- 7.84997610555419e-287


Since all categorical features have pval <=0.5 we will accept all

In [28]:
num_cols = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in['PROSPECTID' ,'Approved_Flag']:
        num_cols.append(i)

In [29]:
len(num_cols)

72

In [30]:
#Anova and mutlicollienarity
#first checking multicollineariity

#vif sequentially check
vif_data = df[num_cols]
total_cols = vif_data.shape[1]
cols_to_kept = []
col_idx = 0

for i in range(0 ,total_cols):
    vif_value = variance_inflation_factor(vif_data ,col_idx)
    print(col_idx ,'---' ,vif_value)
    
    if vif_value <= 6:
        cols_to_kept.append(num_cols[i])
        col_idx += 1
    else:
        vif_data = vif_data.drop([num_cols[i]] ,axis=1)

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000336
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.099653381646731
3 --- 5.5813520096427585
4 --- 1.9855843530987785


C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.3843464059655854
7 --- 3.064658415523423
8 --- 2.898639771299251
9 --- 4.377876915347322
10 --- 2.2078535836958433
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721434


C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427232
15 --- 1.4210050015175733
16 --- 8.083255010190316
16 --- 1.6241227524040112
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.8258570471324314
18 --- 1.5080839450032664
19 --- 2.172088834824577
20 --- 2.623397553527229
21 --- 2.2959970812106176
22 --- 7.360578319196439
22 --- 2.160238777310255
23 --- 2.8686288267891467
24 --- 6.458218003637272
24 --- 2.8474118865638256
25 --- 4.753198156284083
26 --- 16.227354755948223
26 --- 6.424377256363877
26 --- 8.887080381808687
26 --- 2.3804746142952653
27 --- 8.609513476514548
27 --- 13.06755093547673
27 --- 3.500040056654654
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.104855915136433
30 --- 2.7971639638512906
31 --- 3.424171203217696
32 --- 10.175021454450912
32 --- 6.408710354561296
32 --- 1.0011511962625628
33 --- 3.069197305397274
34 --- 2.8091261600643724
35 --- 20.249538381980678
35 --- 15.864576541593774
35 --- 1.8331649740

In [31]:
len(cols_to_kept)

39

In [32]:
#check ANova
from scipy.stats import f_oneway

cols_to_kept_num = []

for i in cols_to_kept:
    a = list(df[i])
    b = list(df['Approved_Flag'])
    
    group_P1= [value for value, group in zip(a ,b) if group == 'P1']
    group_P2= [value for value, group in zip(a ,b) if group == 'P2']
    group_P3= [value for value, group in zip(a ,b) if group == 'P3']
    group_P4= [value for value, group in zip(a ,b) if group == 'P4']
    
    f_stats ,p_val = f_oneway(group_P1 ,group_P2 ,group_P3 ,group_P4)
    if p_val <= 0.05:
        cols_to_kept_num.append(i)

In [33]:
cols_to_kept_num

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag']

In [34]:
len(cols_to_kept_num)

37

In [35]:
df.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3


In [39]:
#Label encoding
cat_cols = []
for i in df.columns:
    if df[i].dtype == 'object':
        cat_cols.append(i)
    
cat_cols

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

In [43]:
for col in cat_cols:
    print(col)
    print(df[col].unique())
    print('-----------')

MARITALSTATUS
['Married' 'Single']
-----------
EDUCATION
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']
-----------
GENDER
['M' 'F']
-----------
last_prod_enq2
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
-----------
first_prod_enq2
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']
-----------
Approved_Flag
['P2' 'P1' 'P3' 'P4']
-----------


In [45]:
# we can assign 1 ,2 ,3 ... to EDUCATION .. so lets OHE this EDUCATION column
df.loc[df['EDUCATION'] == 'SSC' ,['EDUCATION']] = 1
df.loc[df['EDUCATION'] == '12TH' ,['EDUCATION']] = 2
df.loc[df['EDUCATION'] == 'UNDER GRADUATE' ,['EDUCATION']] = 3
df.loc[df['EDUCATION'] == 'GRADUATE' ,['EDUCATION']] = 4

df.loc[df['EDUCATION'] == 'OTHERS' ,['EDUCATION']] = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL' ,['EDUCATION']] = 3

In [51]:
df.loc[df['EDUCATION'] == 'POST-GRADUATE' ,['EDUCATION']] = 4

In [53]:
df['EDUCATION'].value_counts()

EDUCATION
4    16038
2    11703
1     9532
3     4791
Name: count, dtype: int64

In [54]:
df['EDUCATION'].dtype

dtype('O')

In [55]:
df['EDUCATION'] = df['EDUCATION'].astype('int')

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [60]:
df_encoded = pd.get_dummies(df ,columns=['MARITALSTATUS' ,'GENDER' ,'last_prod_enq2' ,'first_prod_enq2'])

In [61]:
df.sample()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
21144,25840,9,6,3,1,2,0.111,0.222,0.333,0.667,...,1.0,0.0,0.5,0.0,1,0,ConsumerLoan,PL,680,P2


In [62]:
df_encoded

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,False,False,True,False,False,False,False,False,True,False
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,True,False,False,False,False,False,True,False,False,False
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,True,False,False,False,False,False,False,False,False,True
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,False,False,False,False,True,False,False,False,False,False
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,True,False,False,False,False,False,True,False,False,False
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,False,False,False,True,False,False,False,False,False,True
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,True,False,False,False,False,False,False,False,False,True
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,True,False,False,False,False,False,False,False,False,True


In [64]:
for i in df_encoded.columns:
    if df_encoded[i].dtype == 'object':
        print("uheh" ,i)
    

uheh Approved_Flag


In [68]:
#model
#1. RandomForest
y = df_encoded['Approved_Flag']
X = df_encoded.drop(['Approved_Flag'] ,axis=1)

In [69]:
X_train ,X_test ,y_train ,y_test = train_test_split(X ,y ,test_size=0.2 ,random_state=42)

In [71]:
X_train.shape ,X_test.shape

((33651, 90), (8413, 90))

In [72]:
rf_classifier = RandomForestClassifier(n_estimators=200 ,random_state=42)

In [73]:
rf_classifier.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [74]:
y_pred = rf_classifier.predict(X_test)

In [75]:
acc = accuracy_score(y_test ,y_pred)

In [76]:
print(acc)

0.9903720432663735


In [77]:
precision ,recall ,f1_score, _ = precision_recall_fscore_support(y_test ,y_pred)

In [78]:
for i, v in enumerate(['p1','p2','p3','p4']):
    print(f"class {v}")
    print(f"precision: {precision[i]}")
    print(f"recall : {recall[i]}")
    print(f"f1_score: {f1_score[i]}")

class p1
precision: 0.9475164011246485
recall : 0.9970414201183432
f1_score: 0.9716482460355599
class p2
precision: 0.9962480252764613
recall : 1.0
f1_score: 0.9981204866950243
class p3
precision: 0.9960222752585521
recall : 0.9449056603773585
f1_score: 0.9697908597986057
class p4
precision: 0.9990243902439024
recall : 0.9951409135082604
f1_score: 0.997078870496592


In [ ]:
#2. XGBoost
